# LSTM MODELİ İLE SINIFLANDIRMA MODELİ OLUŞTURMA #

İlk önce yapacağımız şey gerekli kütüphaneleri import etmek.Burada gerekli olan bütün kütüphaneler var.Sizde yoksa 
indirmeniz gerekecek.

In [1]:
import numpy as np
import pandas as pd
from  sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,GRU,Embedding,CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
import tensorflow as tf

Burada yapacağımız şey elimizde CSV formatında tuttuğumuz veriyi python "DataFrame"sine çevirmek.Python DataFrame özelliğini, bir nevi R dilinden kopyalamıştır.Makina Öğrenmesinde işimize bu DataFrameler çok yarar.Gerekli dökümantasyon: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html 
Bunun için read_csv fonksiyonunu kullanıyoruz.Bu veriyi kullanabileceğimiz bir formata dönüştürür.

In [2]:
total_data = pd.read_csv("data.csv")
bad_review =total_data[total_data["Review"]==0]
#Bu kısmı modelde iyileştirmeye gitmek için yaptım. --->
X_ = bad_review["Review"]
y_=bad_review["Rating"]
#Buraya kadar <---
total_data.size

486994

Göründüğü gibi elimizde toplam 486.994 adet sınıflandırılmış veri var.Bu proje için gayet yeterli bir veri miktarı.

In [3]:
total_data.head()

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [ ]:
total

Şimdi veriye baktığımızda elimizde 2 sütunumuz var.Biri skorlar, diğeri puanlanan cümleler.Bizim bunları ayırmamız gerekiyor o yüzden X ve y adında 2 adet değişken tanımlayacağız.  **X'e cümleleri ,y'ye skorları koyacağız.**

In [151]:
X__= total_data["Review"].values
X= np.append(X__,X_)
y__=total_data["Rating"].values
y= np.append(y__,y_)

Bir önceki satırda veriyi skorlar ve cümleler olarak ikiye ayırmıştık.Şimdi makine öğrenmesinde çok önemli olan bir diğer özelliğe geldik. **Veriyi Train-Test olarak ayırma.** Bunu yapmamızın nedeni verinin doğruluk oranını eğittiğimiz veriden bağımsız bir şekilde ölçmek.Bunun eğitilmiş veriden bağımsız olması gerekiyor ki bize doğru bir sonuç versin.Bunun için **train_test_split** adında fonksiyon kullanacağız.Bu fonksiyon veriyi en verimli şekilde ayırır.Elimizdeki verinin %20'sini Test için ayıracağız.

In [159]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20,random_state=41)


Şimdi veri ikiye bölünmüş durumda.Şuan yapacağımız şey veriyi **string formatından int formatına çevirmek.Bunu yapmamızın nedeni yapay sinir ağlarını sadece sayısal bir şekilde eğitebilmemizdir.Bunun için gerekli fonksiyonlar aşağıda.** num_words değişkenine de 10000 dememizin nedeni sadece en çok geçen 10bin kelimeyi alıp,gerisini yoksaymak istememizdir.Veride nadir kullanılan kelimelerin etkisini yoksaymak için bunu kullanmamız gerekiyor.

In [160]:
num_words=10000
tokenizer =Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X)

Önceki satırda tokenizer adında bir sayısallaştırıcı fonksiyon oluşturduk.Bu fonksiyon sayesinde verileri string tipinden int tipine dönüştürebiliriz.Elimizdeki X_train ve X_test verilerine öyle yapacağız.Bu yaptığımız işleme **Tokenleştirme** denmektedir.

In [161]:
X_train_tokens=tokenizer.texts_to_sequences(X_train)
X_test_tokens=tokenizer.texts_to_sequences(X_test)

Şimdi yapacağımız şey bir cümlenin en fazla ne kadar uzun olabileceğini seçmek.Bunu kafamızdan da sallayabiliriz ama bu fonksiyonel olmaz.Elimizdeki veri durumuna göre hesaplama yapmamız lazım.

In [162]:
num_tokens=[len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens=np.array(num_tokens)
max_tokens= np.mean(num_tokens)+ np.std(num_tokens)*2
max_tokens=int(max_tokens)

X_train_pad =pad_sequences(X_train_tokens,maxlen=max_tokens)
X_test_pad =pad_sequences(X_test_tokens,maxlen=max_tokens)
max_tokens

59

Bu verisetine göre en iyi sonucu 59 karaktere kadar sınırlaştırırsak alırız.Cümle sayısı 59'dan küçük olan cümlelerin sonuna 0 koyar, büyük olan cümlelerin de 59'dan sonrasını siler.Bu bizim modelimizi daha optimal yapar hem de gereksiz uzunluktan bizi kurtarır.

# Şimdi sıra model oluşturmada #

In [176]:
model = Sequential()
embedding_size = 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name="embedding_layer"))
#Burada LSTM modelinin bir türü olan GRU'yu kullanacağız.İkisinden de aynı değerleri alırız fakat GRU daha optimize çalışır.
#Önceki tespitlerime göre 2 derin ağ 16-8 olarak bu veride en optimal çalışıyor.O yüzden 3 katman ekliyoruz.
model.add(GRU(units=16,return_sequences=True))
model.add(GRU(units=8))
"""Şimdi derin ağlarımızı oluşturduk.Çıkış ağına da sigmoid =(1/(1+e^(-x) )) fonksiyonu kullanıyoruz.
Bu fonksiyon gelen değerleri 0 ile 1 olarak döndürür.Yani en yüksek tahmin skorlarını 1'e yaklaştırır.
En düşüklerini 0'a.Bu özellik te bize cümlelerini sınıflandırma şansı verir. """

model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
               metrics=["accuracy"])
#Optimizer = Geriye yayılım sırasında kullanılacak fonksiyon.Bu modelin parametrelerinin en iyi şekilde olmasına yarıyor.
#metrics = Doğruluğu %sel olarak gösterecek fonksiyon.


In [177]:
model.summary() #Model hakkında tüm bilgi burada yazar.

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
gru_21 (GRU)                 (None, 59, 16)            3216      
_________________________________________________________________
gru_22 (GRU)                 (None, 8)                 600       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 503,825
Trainable params: 503,825
Non-trainable params: 0
_________________________________________________________________


# Şimdi sıra modelimizi eğitmekte

Epoch 'un anlamı aynı veriyi modele tekrar tekrar vermektir.Bu öğrenmeyi iyi bir şekilde arttırır.Fakat bazı sorunları vardır.Sayıyı çok tutarsak **overfitting (aşırı öğrenme)** dediğimiz bir sorun ortaya çıkarır.O yüzden 3 bu verinin çapına göre en iyi rakamdır.İyi bir öğrenme sunar hem de modelin başarıyla çalışmasını sağlar.

In [178]:
model.fit(X_train_pad,y_train,epochs=3,batch_size=256)

Train on 194797 samples
Epoch 1/3
194797/194797 [==============================] - 98s 504us/sample - loss: 0.1529 - accuracy: 0.9539
Epoch 2/3
194797/194797 [==============================] - 97s 499us/sample - loss: 0.0917 - accuracy: 0.9681
Epoch 3/3
194797/194797 [==============================] - 98s 501us/sample - loss: 0.0762 - accuracy: 0.9739


# Modelimizi eğittik.İlk bakışta %97.3 başarı oranı, % 0.76 kayıp oranımız var.Bu çok iyi bir değer.Şimdi test için ayırdığımız veride de başarı oranımızı hesaplayacağız.

In [166]:
model.evaluate(X_test_pad,y_test)

48700/48700 [==============================] - 33s 670us/sample - loss: 0.1092 - accuracy: 0.9646


[0.10919102820986595, 0.96455854]

In [ ]:
y_train, 95 = 1 ,5 =0 y_tahmin 93=1, 7=0

Eğittiğimiz veriden bağımsız doğruluk sonucumuz da % 96 .Şimdi 1-2 yorum denemesi yapalım.Bunun için bir fonksiyon yapacağız.

In [181]:
def tahmin(cumleler):
    tokens=tokenizer.texts_to_sequences(cumleler)
    tokens_pad = pad_sequences(tokens,maxlen=max_tokens)
    tahmin =model.predict(tokens_pad)
    print(tahmin)
    if tahmin>0.45 and tahmin<0.65:
        print("Bu cümle nötrdür.")
    elif tahmin <0.40 :
        print("Bu cümle olumsuzdur.")
    elif tahmin > 0.60 :
        print("Bu cümle olumludur.")

In [249]:
cumle = ["Bu kadar dandik bi uygulama. Bu kadar zor ve anlaşılması imkansız bi saçma uygulama görmedim. Yahu bi mobil uygulamada kredi kartı başvurusunu yapamadım saatlerdir uğraşıyorumş"]
tahmin(cumle)

[[0.2997198]]
Bu cümle olumsuzdur.


In [250]:
cumle =["Pek benlik değil ama gene de iyi"]
tahmin(cumle)

[[0.91844386]]
Bu cümle olumludur.


In [252]:
cumle =["Çok kalitesiz duruyor"]
tahmin(cumle)

[[0.05859238]]
Bu cümle olumsuzdur.


In [253]:
cumle =["Uygulamadan gayet memnun kaldım"]
tahmin(cumle)

[[0.99264973]]
Bu cümle olumludur.


# Şimdi model ağırlıklarını kaydedelim

In [251]:
model.save("review_classification.h5")

# *SON*